### 14. visdom

- pip install visdom 
- -> cmd : python -m visdom.server 
    - or Jupyter Notebook > Terminal 를 새로 켜서 python -m visdom.server 입력
- -> web browser : http://localhost:8097

In [1]:
import torch 
import torch.nn as nn 
import torchvision 
import torchvision.datasets as dsets
import torchvision.transforms as transforms 

In [2]:
import visdom 
vis = visdom.Visdom()

Setting up a new session...


In [3]:
## text 
vis.text('Hello World!', env = 'main')

'window_39493a7666581c'

In [4]:
## image ~ RGB 200x200의 랜덤이미지 생성 
## vis.image() 함수를 사용하고, 인자로 (C,H,W) 형태의 텐서를 입력
a = torch.randn(3, 200, 200)
vis.image(a)

'window_39493a76739ea6'

In [5]:
# vis.images를 쓰면 복수의 이미지가 생성된
# (batch_size, C, H, W) 형태의 텐서를 입력
vis.images(torch.Tensor(3,3,28,28))

'window_39493a768fd94e'

- MNIST와 CIFAR10 데이터를 가져와 시각화 해보자 

In [6]:
## load the datasets 
MNIST = dsets.MNIST(root = 'MNIST_data/',
                    train = True, 
                    transform = transforms.ToTensor(), 
                    download = True)

cifar10 = dsets.CIFAR10(root = 'CIFAR10_data/',
                        train = True, 
                        transform = transforms.ToTensor(), 
                        download = True)

Files already downloaded and verified


In [7]:
##__getitem__함수로 0번째 이미지를 가져와서 이미지를 표시
data1 = MNIST.__getitem__(0)
print(data1[0].shape)
vis.images(data1[0], env="main")

data2 = cifar10.__getitem__(0)
print(data2[0].shape)
vis.images(data2[0], env="main")

torch.Size([1, 28, 28])
torch.Size([3, 32, 32])


'window_39493a7751549c'

In [8]:
### 32개의 데이터(이미지)를 한번에 확인하기 
data_loader = torch.utils.data.DataLoader(dataset = MNIST, 
                                         batch_size = 32, 
                                         shuffle = False)

for num, value in enumerate(data_loader) : 
    value = value[0]
    print(value.shape)
    vis.images(value)
    break 

torch.Size([32, 1, 28, 28])


In [9]:
### clean the window 
vis.close(env = 'main')

''

- Line plot을 그려보자 

In [10]:
## X와 Y의 shape가 일치해야 한다. 
## if X를 지정하지 않으면 자동으로 0~1로 X축이 만들어진다. 
## cf. rand()는 0~1 사이의 난수 생성, randn()은 normal distribution(0, 1)에 따른 난수 생성 

X_data = torch.Tensor([1,2,3,4,5])
Y_data = torch.randn(5)
plt = vis.line(X=X_data, Y= Y_data)

In [11]:
## append 함수로 기존 차트에 값을 추가 
X_append = torch.Tensor([6])
Y_append = torch.randn(1)

## updata 대상 창을 지정해 줘야 한다. (win= plt)
vis.line(X= X_append, Y= Y_append, update= 'append', win= plt)

'window_39493a77818c3a'

In [12]:
### 다수의 라인 그리고, legend 붙이기 
num = torch.Tensor(list(range(0,10)))
num = num.view(-1,1)
num = torch.cat((num,num),dim=1)
plt = vis.line(Y=torch.randn(10,2), X = num)

## opts 함수에 딕셔너리 값을 주어 legend 생성 
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

- 학습의 loss를 나타내는 그래프를 그리는 loss_tracker를 만들어보자. loss가 나올때마다 epoch와 loss 값을 각각 X, Y값으로 받아 그래프에 갱신한다. 이를 통해 loss가 어떻게 변하는지 시각적으로 살펴볼 수 있다. 

In [13]:
## 함수 
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [14]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [15]:
vis.close(env='main')

''

- MNIST CNN에 visdom 적용하기 code
    - loss_tracker함수 정의 > blank window 생상 > training 코드에 loss_tracker 추가 

In [16]:
import torch 
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
import random 
import matplotlib.pyplot as plt 
import timeit 
device = 'cpu'
import visdom 
vis = visdom.Visdom()
vis.close(env="main")  ##기존 윈도우가 있으면 모두 비운다 

## parameters 
training_epochs = 20 
batch_size = 100 
learning_rate = 0.001 

## dataset and DataLoader
mnist_train = dsets.MNIST(root = 'MNIST_data/', 
                          train = True, 
                          transform = transforms.ToTensor(), 
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', 
                          train = False, 
                          transform = transforms.ToTensor(), 
                          download = True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           drop_last = True ) 

Setting up a new session...


In [17]:
## loss tracker 
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [18]:
### CNN 3 layers 

class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [19]:
## model, loss F, optimizer 
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

torch.Size([1, 10])


In [20]:
## make blank plot !!
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),
                    opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [21]:
total_batch = len(data_loader)
start_time = timeit.default_timer()  ## start time check 

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    
end_time = timeit.default_timer()  ## end time check 
print('Running time :', end_time - start_time)

[Epoch:1] cost = 0.16122494637966156
[Epoch:2] cost = 0.04343816265463829
[Epoch:3] cost = 0.02823657914996147
[Epoch:4] cost = 0.02178046479821205
[Epoch:5] cost = 0.017979910597205162
[Epoch:6] cost = 0.014513688161969185
[Epoch:7] cost = 0.013352546840906143
[Epoch:8] cost = 0.011301113292574883
[Epoch:9] cost = 0.009277759119868279
[Epoch:10] cost = 0.008520971983671188
[Epoch:11] cost = 0.009094811044633389
[Epoch:12] cost = 0.006747728679329157
[Epoch:13] cost = 0.006259240210056305
[Epoch:14] cost = 0.005833561532199383
[Epoch:15] cost = 0.0046035172417759895
[Epoch:16] cost = 0.00483482563868165
[Epoch:17] cost = 0.004873053636401892
[Epoch:18] cost = 0.005344661884009838
[Epoch:19] cost = 0.004065488465130329
[Epoch:20] cost = 0.00503891333937645
Running time : 1982.1961023


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

Accuracy: 0.9853000044822693
